In [194]:
import pandas as pd
import json
import requests

In [195]:
# This method uses the canadian governments free geolocation service to extract matching coordinates based on 
# provided address
def get_address_coords(address):
    # 1400 12 Ave SW, Calgary, AB T3C 0P7, 51.043345859995114, -114.0940433180538
    addressDF = requests.get('https://geogratis.gc.ca/services/geolocation/en/locate?q=' + address)
    userAddressList = addressDF.json()

    for x in userAddressList[1].values():
        if type(x) is dict:
            return x.get('coordinates')
            
def read_data_from_csv(filename):
    # Logic to read data from CSV
    dataFrame = pd.read_csv(filename)
    return dataFrame

# converts lat and long into km
def convert_coord_km(lat, long):
    return abs(lat*111.2) + abs(long*111.3)

# Filters the dataFrame by a desired parking restriction in the csv
def filter_parking_restrictions(dataFrame, desiredRestriction):
    for x in dataFrame.index:
        if dataFrame.loc[x,"PARKING_RESTRICTION"] != desiredRestriction:
            dataFrame.drop(x, inplace = True)
    return dataFrame


In [196]:
# print(type(get_address_coords('1400 12 Ave SW, Calgary')))
userCoords = get_address_coords('1400 12 Ave SW, Calgary')
onsDF = read_data_from_csv('/Users/auz/HackathonStuff/Parking-Locator/backend/datasets/On-Street_Parking_Zones_20240217.csv')
resDF = read_data_from_csv('/Users/auz/HackathonStuff/Parking-Locator/backend/datasets/On-Street_Residential_Parking_Zones_20240217.csv')

# Parses the dataframe for coords and returns a list of lists containing floats for further analysis
def get_float_coords(dataFrame):
    coordsList = []
    for string in resDF.loc[:, "line"]:
        getvals = []
        for val in string:
            if val.isdigit() or val.isspace() or val == '.':
                getvals.append(val)
        result = "".join(getvals)
        result = result.strip()
        list2 = result.split(" ")
        listnum = []
        for s in list2:
            z = float(s)
            listnum.append(z)
        coordsList.append(listnum)
    return coordsList

print(get_float_coords(resDF))

# for string in coordsList:
#     # print(type(string))
#     string.split(",")
#     # print(string)

[[114.098641, 51.0575562, 114.0987358, 51.057556, 114.0989327, 51.0575555, 114.0991296, 51.057555, 114.0992484, 51.0575548, 114.0992498, 51.0575548], [114.1213625, 51.0820441, 114.1231883, 51.0825275], [114.0455034, 51.0377927, 114.045505, 51.0377927, 114.0455774, 51.0377925, 114.0457756, 51.037792, 114.0459285, 51.0377916], [114.1233199, 51.0825637, 114.1233826, 51.0825809, 114.1234352, 51.0825947, 114.123447, 51.0825978], [114.0450259, 51.0377927, 114.0452942, 51.0377924], [114.1202903, 51.0817056, 114.1204806, 51.081799, 114.1205208, 51.0818137, 114.1205585, 51.081827, 114.1206049, 51.0818422, 114.1206806, 51.0818639, 114.1207646, 51.0818852, 114.1208612, 51.0819107, 114.1209332, 51.0819304, 114.1209872, 51.0819447, 114.1213625, 51.0820441], [114.1235466, 51.0826239, 114.1236092, 51.0826411, 114.1236642, 51.0826579, 114.1237248, 51.0826825, 114.1237667, 51.0826965, 114.1237967, 51.0827111, 114.123841, 51.0827354, 114.1241137, 51.0829043], [114.0807993, 51.0510765, 114.0808061, 51.05

In [197]:
# Parses a list of lists of floats to produce the avg lat and long of lines
def avg_lat_long(coordsList):
    tuplesList = []
    for li in coordsList:
        l2 = []
        li.sort()
        sumFirstHalf = 0
        sumSecondHalf = 0
        for i in range(len(li)):
            if i < len(li)/2:
                sumFirstHalf += li[i]
                # print(sumFirstHalf)
            else:
                sumSecondHalf += li[i]
        l2.append(sumFirstHalf/(len(li)/2))
        l2.append(sumSecondHalf/(len(li)/2))
        tuplesList.append(l2)
    return tuplesList


tuplesList = avg_lat_long(get_float_coords(resDF))

In [198]:
# takes in userCoords and tuplesList to generate a list of distances from the user coords
def compare_distances(userCoords, tuplesList):
    distanceFromUserList = []
    for li in tuplesList:
        lat = abs(userCoords[1] - li[0])
        long = abs(li[1] + userCoords[0])
        distanceFromUserList.append(convert_coord_km(lat=lat, long=long))
    print(distanceFromUserList)
    return distanceFromUserList


distanceLi = compare_distances(userCoords=userCoords, tuplesList=tuplesList)        
print(resDF)


[2.5512017116642367, 7.892939149999687, 5.582256895998683, 8.05087182749945, 5.637648705001309, 7.674809490001131, 8.104728796247885, 2.1604021600013015, 1.4004438500009442, 2.176370576667952, 4.629682930000166, 4.400271350000376, 4.811562975883926, 4.534364373229569, 4.494335524001408, 2.5152555299988038, 2.3728414399993327, 2.2954683450006272, 2.435208069998474, 5.748014005000916, 8.021620999165837, 5.75631458999823, 2.7624338650015985, 2.1196841700002196, 5.734539540000304, 8.038237385834353, 3.674423033333996, 3.6309089949994284, 5.704290647670811, 2.136821512499067, 2.2046102579993074, 7.6312958949990035, 5.624194140000822, 13.995808568334972, 13.873715643332057, 13.845548426666937, 13.79233281000108, 13.97179135500092, 13.998774008752342, 8.24450577666527, 8.435615155456134, 8.309004784286408, 8.312719982500155, 8.450496256664994, 5.01150445333297, 6.1233161249994446, 6.465182544000605, 6.262536649999065, 6.142410719999293, 6.06852773999795, 6.663046884998803, 32.72356159772574, 

In [199]:
# filters dataframe based on distanceList got from compare_distances method and a desired distance filter
def filter_df_distance(df, distanceList, desiredDistance):
    for i in range(len(distanceList)):
        if distanceList[i] > desiredDistance:
            df.drop(index=i, inplace=True)
    return df
filter_df_distance(resDF, distanceLi, .5)

# print(resDF)

,DESCRIPTION,PARKING_ZONE,PERMIT_ZONE,STALL_TYPE,STATUS,ADDRESS_DESC,CTP_CLASS,DOT,ENFORCEABLE_TIME,NO_PARKING,NO_STOPPING,PARKING_RESTRICTION,TIME_RESTRICTION,CAMERA,OCTANT,line,Global_ID
402,R3395,P,P,Parallel,Active,1200 Block 13 AV SW,Residential Street,E,0700-1700 MON-FRI,NaN,NaN,120,0700-1700 MON-FRI,FR,SW,"MULTILINESTRING ((-114.0901496 51.0411951, -11...",689ae1e8-dce0-4865-a5f7-fddd2199d56e
515,R6883,P,P,Parallel,Active,1200 Block 13 AV SW,Residential Street,E,0700-1700 MON-FRI,NaN,NaN,120,0700-1700 MON-FRI,FR,SW,"MULTILINESTRING ((-114.0894216 51.0411746, -11...",609043cd-4247-4353-acb8-79421df64f2d
591,R3392,P,P,Parallel,Active,1300 Block 15 AV SW,Residential Street,E,0700-1800 MON-FRI,NaN,NaN,120,0700-1800 MON-FRI,FR,SW,"MULTILINESTRING ((-114.0918078 51.0393435, -11...",715147a8-b286-45bc-8b52-ebe7d5e47e97
632,R4336,C,C,Parallel,Active,1100 Block 13 AV SW,Residential Street,E,0700-1800 MON-FRI,NaN,NaN,120,0700-1800 MON-FRI,FR,SW,"MULTILINESTRING ((-114.088084 51.0411377, -114...",879c93a5-020a-425b-96af-3bb42929396c
2453,R3393,P,P,Parallel,Active,1500 Block 12 ST SW,Residential Street,S,0700-1800 MON-FRI,NaN,NaN,120,0700-1800 MON-FRI,FR,SW,"MULTILINESTRING ((-114.0916632 51.0390218, -11...",7aa9ed47-326e-4358-a6f5-3cdd06536e10
2534,R7898,P,P,Parallel,Active,1300 Block 11 ST SW,Residential Street,S,0700-1800 MON-FRI,NaN,NaN,180,0700-1800 MON-FRI,FR,SW,"MULTILINESTRING ((-114.089053 51.0409898, -114...",42d456b6-fc36-4fb4-919d-593227b5c621
3410,R3402,P,P,Parallel,Active,1300 Block 15 AV SW,Residential Street,W,0700-1800 MON-FRI,NaN,NaN,120,0700-1800 MON-FRI,FR,SW,"MULTILINESTRING ((-114.0932025 51.0394658, -11...",a41b949f-f590-42d5-8989-b00e2e08bbfd
